### Import Libraries

In [1]:
#import OpenCV
import cv2

#import pickle
import pickle

#import tensorflow
import tensorflow as tf
import tensorflow_hub as hub

#import numpy, pandas and m
import numpy as np
import pandas as pd
import math as m

#import os
import os

#import csv
import csv

import warnings
warnings.filterwarnings('ignore')

### Load Tensorflow Movenet Model

In [2]:
model = hub.load('https://tfhub.dev/google/movenet/multipose/lightning/1')
movenet = model.signatures['serving_default']

### Get and Draw Keypoints

In [3]:
def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 4, (0,255,0), -1) 

In [4]:
def label(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.putText(frame, (int(kx), int(ky)), 4, (0,255,0), -1)

### Draw Edges and Connections

In [5]:
EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

In [6]:
def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 2)

In [7]:
# Function to loop through each person detected and render
def loop_through_people(frame, keypoints_with_scores, edges, confidence_threshold):
    for person in keypoints_with_scores:
        draw_connections(frame, person, edges, confidence_threshold)
        draw_keypoints(frame, person, confidence_threshold)

### Make Detections with Model

In [8]:
#import the pickled rf model
with open('../Data/movenet_rf_model_left_right_dr_dl_3.pkl', 'rb') as f:
    movenet_model = pickle.load(f)

### Establishing Functions for Angle Calculations

In [9]:
#To get joint angles through 3 points on a body
def calculate_angle(a,b,c):
    a = np.array(a) # First point on a body
    b = np.array(b) # Mid point on a body
    c = np.array(c) # End point on a body
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle > 180.0:
        angle = 360.0-angle
    return angle

In [10]:
#To get neck inclination angles
def calculate_angle2(x1, y1, x2, y2):
    theta = m.acos((y2-y1)*(-y1)/ (m.sqrt((x2 - x1)**2 + (y2-y1)**2) *y1))
    degree = int(180/m.pi)* theta
    return degree

In [11]:
#Use IP Webcam app (1920x1080 resolution)
cap = cv2.VideoCapture('http://192.168.1.23:8080/video')
while cap.isOpened():
    ret, frame = cap.read()
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    # Resize image
    img = frame.copy()
    img = tf.image.resize_with_pad(tf.expand_dims(img, axis=0), 192,320)
    input_img = tf.cast(img, dtype=tf.int32)
    
    # Detection section
    results = movenet(input_img)
    keypoints_with_scores = results['output_0'].numpy()[:,:,:51].reshape((6,17,3))
    
    try:
        left_shoulder_x = keypoints_with_scores[0][5][1]
        left_shoulder_y = keypoints_with_scores[0][5][0]
        left_ear_x = keypoints_with_scores[0][3][1]
        left_ear_y = keypoints_with_scores[0][3][0]
        
        right_shoulder_x = keypoints_with_scores[0][6][1]
        right_shoulder_y = keypoints_with_scores[0][6][0]
        right_ear_x = keypoints_with_scores[0][4][1]
        right_ear_y = keypoints_with_scores[0][4][0]
        
        left_shoulder = [left_shoulder_x, left_shoulder_y]
        
        left_hip = [keypoints_with_scores[0][11][1], keypoints_with_scores[0][11][0]]
        left_elbow = [keypoints_with_scores[0][7][1], keypoints_with_scores[0][7][0]]
        left_knee = [keypoints_with_scores[0][13][1], keypoints_with_scores[0][13][0]]
        left_ankle = [keypoints_with_scores[0][15][1], keypoints_with_scores[0][15][0]]
        left_wrist = [keypoints_with_scores[0][9][1], keypoints_with_scores[0][9][0]]
        left_ear = [keypoints_with_scores[0][3][1], keypoints_with_scores[0][3][0]]
        
        right_hip = [keypoints_with_scores[0][12][1], keypoints_with_scores[0][12][0]]
        right_shoulder = [keypoints_with_scores[0][6][1], keypoints_with_scores[0][6][0]]
        right_elbow = [keypoints_with_scores[0][8][1], keypoints_with_scores[0][8][0]]
        right_knee = [keypoints_with_scores[0][14][1], keypoints_with_scores[0][14][0]]
        right_ankle = [keypoints_with_scores[0][16][1], keypoints_with_scores[0][16][0]]
        right_wrist = [keypoints_with_scores[0][10][1], keypoints_with_scores[0][10][0]]
        right_ear = [keypoints_with_scores[0][5][1], keypoints_with_scores[0][5][0]]
        
        neck_inclination = calculate_angle2(left_shoulder_x, left_shoulder_y, left_ear_x, left_ear_y)
        
        left_hip_angle = calculate_angle(left_shoulder, left_hip, left_knee)
        #left_knee_angle = calculate_angle(left_hip, left_knee, left_ankle)
        left_bicep_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
        
        right_hip_angle = calculate_angle(right_shoulder, right_hip, right_knee)
        #right_knee_angle = calculate_angle(right_hip, right_knee, right_ankle)
        right_bicep_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)
        #print(left_bicep_angle)
        #print(neck_inclination)
        #print(left_shoulder)
        cv2.putText(frame , str(int(neck_inclination)), tuple(np.multiply(left_shoulder, [1850,800]).astype(int)), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255), 2, cv2.LINE_AA)
        cv2.putText(frame, str(int(left_hip_angle)), tuple(np.multiply(left_hip, [1850,1100]).astype(int)), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255), 2, cv2.LINE_AA)
        #cv2.putText(frame, str(int(left_knee_angle)), tuple(np.multiply(left_knee, [1850,800]).astype(int)), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255), 2, cv2.LINE_AA)
        cv2.putText(frame, str(int(left_bicep_angle)), tuple(np.multiply(left_elbow, [1850,1100]).astype(int)), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255), 2, cv2.LINE_AA)
    except:
        pass
    
    #Export coordinates

    # Render keypoints 
    loop_through_people(frame, keypoints_with_scores, EDGES, 0.3)
    
    cv2.imshow('Movenet Multipose', frame)
    
    if cv2.waitKey(10) & 0xFF==ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

In [73]:
cap = cv2.VideoCapture('http://192.168.1.23:8080/video')
while cap.isOpened():
    ret, frame = cap.read()
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    # Resize imageq
    img = frame.copy()
    img = tf.image.resize_with_pad(tf.expand_dims(img, axis=0), 192,320)
    input_img = tf.cast(img, dtype=tf.int32)
    
    # Detection section
    results = movenet(input_img)
    keypoints_with_scores = results['output_0'].numpy()[:,:,:51].reshape((6,17,3))
    
    try:
        left_shoulder_x = keypoints_with_scores[0][5][1]
        left_shoulder_y = keypoints_with_scores[0][5][0]
        left_ear_x = keypoints_with_scores[0][3][1]
        left_ear_y = keypoints_with_scores[0][3][0]
        
        right_shoulder_x = keypoints_with_scores[0][6][1]
        right_shoulder_y = keypoints_with_scores[0][6][0]
        right_ear_x = keypoints_with_scores[0][4][1]
        right_ear_y = keypoints_with_scores[0][4][0]
        
        left_shoulder = [left_shoulder_x, left_shoulder_y]
        
        left_hip = [keypoints_with_scores[0][11][1], keypoints_with_scores[0][11][0]]
        left_elbow = [keypoints_with_scores[0][7][1], keypoints_with_scores[0][7][0]]
        left_knee = [keypoints_with_scores[0][13][1], keypoints_with_scores[0][13][0]]
        left_ankle = [keypoints_with_scores[0][15][1], keypoints_with_scores[0][15][0]]
        left_wrist = [keypoints_with_scores[0][9][1], keypoints_with_scores[0][9][0]]
        left_ear = [keypoints_with_scores[0][3][1], keypoints_with_scores[0][3][0]]
        
        right_hip = [keypoints_with_scores[0][12][1], keypoints_with_scores[0][12][0]]
        right_shoulder = [keypoints_with_scores[0][6][1], keypoints_with_scores[0][6][0]]
        right_elbow = [keypoints_with_scores[0][8][1], keypoints_with_scores[0][8][0]]
        right_knee = [keypoints_with_scores[0][14][1], keypoints_with_scores[0][14][0]]
        right_ankle = [keypoints_with_scores[0][16][1], keypoints_with_scores[0][16][0]]
        right_wrist = [keypoints_with_scores[0][10][1], keypoints_with_scores[0][10][0]]
        right_ear = [keypoints_with_scores[0][5][1], keypoints_with_scores[0][5][0]]
        
        neck_inclination = calculate_angle2(left_shoulder_x, left_shoulder_y, left_ear_x, left_ear_y)
        
        left_hip_angle = calculate_angle(left_shoulder, left_hip, left_knee)
        #left_knee_angle = calculate_angle(left_hip, left_knee, left_ankle)
        left_bicep_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
        
        right_hip_angle = calculate_angle(right_shoulder, right_hip, right_knee)
        #right_knee_angle = calculate_angle(right_hip, right_knee, right_ankle)
        right_bicep_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)
        #print(left_bicep_angle)
        #print(neck_inclination)
        #print(left_shoulder)
    except:
        pass
    
    #Export coordinates
    try:
        pose_row =list(keypoints_with_scores[0].flatten())
        
        left_bicep_row = list(np.array(left_bicep_angle).flatten())
        left_hip_row = list(np.array(left_hip_angle).flatten())
        
        right_bicep_row = list(np.array(right_bicep_angle).flatten())
        right_hip_row = list(np.array(right_hip_angle).flatten())
        
        neck_inclination_row = list(np.array(neck_inclination).flatten())
        
        row = neck_inclination_row + left_hip_row + left_bicep_row + right_hip_row + right_bicep_row + pose_row
        #row.insert(0,class_name)
        
        #with open('coords_movenet_lieback.csv', mode ='a', newline ='') as f:
                #csv_writer = csv.writer(f, delimiter =',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                #csv_writer.writerow(row)
                
    #Make Detections and Predictions
        X = pd.DataFrame([row])
        movenet_class = movenet_model.predict(X)[0]
        movenet_prob = movenet_model.predict_proba(X)[0]
        #print(movenet_class, movenet_prob)
        
    # Get status box
        cv2.rectangle(frame, (0,0), (250, 60), (245, 117, 16), -1)
        
    # Display Class
        cv2.putText(frame, 'CLASS'
                    , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(frame, movenet_class.split(' ')[0]
                    , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
    # Display Probability
        cv2.putText(frame, 'PROB'
                    , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(frame, str(round(movenet_prob[np.argmax(movenet_prob)],2))
                    , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)   
        
    #Visualise
        cv2.putText(frame , str(int(neck_inclination)), tuple(np.multiply(left_shoulder, [1850,800]).astype(int)), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255), 2, cv2.LINE_AA)
        cv2.putText(frame, str(int(left_hip_angle)), tuple(np.multiply(left_hip, [1850,1100]).astype(int)), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255), 2, cv2.LINE_AA)
        #cv2.putText(frame, str(int(left_knee_angle)), tuple(np.multiply(left_knee, [1850,800]).astype(int)), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255), 2, cv2.LINE_AA)
        cv2.putText(frame, str(int(left_bicep_angle)), tuple(np.multiply(left_elbow, [1850,1100]).astype(int)), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255), 2, cv2.LINE_AA)
    except:
        pass
    
    # Render keypoints 
    loop_through_people(frame, keypoints_with_scores, EDGES, 0.3)
    
    cv2.imshow('Movenet Multipose', frame)
    
    if cv2.waitKey(10) & 0xFF==ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

In [12]:
cap = cv2.VideoCapture('http://192.168.1.23:8080/video')
while cap.isOpened():
    ret, frame = cap.read()
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    # Resize imageq
    img = frame.copy()
    img = tf.image.resize_with_pad(tf.expand_dims(img, axis=0), 192,320)
    input_img = tf.cast(img, dtype=tf.int32)
    
    # Detection section
    results = movenet(input_img)
    keypoints_with_scores = results['output_0'].numpy()[:,:,:51].reshape((6,17,3))
    
    try:
        left_shoulder_x = keypoints_with_scores[0][5][1]
        left_shoulder_y = keypoints_with_scores[0][5][0]
        left_ear_x = keypoints_with_scores[0][3][1]
        left_ear_y = keypoints_with_scores[0][3][0]
        
        right_shoulder_x = keypoints_with_scores[0][6][1]
        right_shoulder_y = keypoints_with_scores[0][6][0]
        right_ear_x = keypoints_with_scores[0][4][1]
        right_ear_y = keypoints_with_scores[0][4][0]
        
        left_shoulder = [left_shoulder_x, left_shoulder_y]
        
        left_hip = [keypoints_with_scores[0][11][1], keypoints_with_scores[0][11][0]]
        left_elbow = [keypoints_with_scores[0][7][1], keypoints_with_scores[0][7][0]]
        left_knee = [keypoints_with_scores[0][13][1], keypoints_with_scores[0][13][0]]
        left_ankle = [keypoints_with_scores[0][15][1], keypoints_with_scores[0][15][0]]
        left_wrist = [keypoints_with_scores[0][9][1], keypoints_with_scores[0][9][0]]
        left_ear = [keypoints_with_scores[0][3][1], keypoints_with_scores[0][3][0]]
        
        right_hip = [keypoints_with_scores[0][12][1], keypoints_with_scores[0][12][0]]
        right_shoulder = [keypoints_with_scores[0][6][1], keypoints_with_scores[0][6][0]]
        right_elbow = [keypoints_with_scores[0][8][1], keypoints_with_scores[0][8][0]]
        right_knee = [keypoints_with_scores[0][14][1], keypoints_with_scores[0][14][0]]
        right_ankle = [keypoints_with_scores[0][16][1], keypoints_with_scores[0][16][0]]
        right_wrist = [keypoints_with_scores[0][10][1], keypoints_with_scores[0][10][0]]
        right_ear = [keypoints_with_scores[0][5][1], keypoints_with_scores[0][5][0]]
        
        neck_inclination = calculate_angle2(left_shoulder_x, left_shoulder_y, left_ear_x, left_ear_y)
        
        left_hip_angle = calculate_angle(left_shoulder, left_hip, left_knee)
        #left_knee_angle = calculate_angle(left_hip, left_knee, left_ankle)
        left_bicep_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
        
        right_hip_angle = calculate_angle(right_shoulder, right_hip, right_knee)
        #right_knee_angle = calculate_angle(right_hip, right_knee, right_ankle)
        right_bicep_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)
        #print(left_bicep_angle)
        #print(neck_inclination)
        #print(left_shoulder)
    except:
        pass
    
    #Export coordinates
    try:
        pose_row =list(keypoints_with_scores[0].flatten())
        
        left_bicep_row = list(np.array(left_bicep_angle).flatten())
        left_hip_row = list(np.array(left_hip_angle).flatten())
        
        right_bicep_row = list(np.array(right_bicep_angle).flatten())
        right_hip_row = list(np.array(right_hip_angle).flatten())
        
        neck_inclination_row = list(np.array(neck_inclination).flatten())
        
        row = neck_inclination_row + left_hip_row + left_bicep_row + right_hip_row + right_bicep_row + pose_row
        #row.insert(0,class_name)
        
        #with open('coords_movenet_lieback.csv', mode ='a', newline ='') as f:
                #csv_writer = csv.writer(f, delimiter =',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                #csv_writer.writerow(row)
                
    #Make Detections and Predictions
        X = pd.DataFrame([row])
        movenet_class = movenet_model.predict(X)[0]
        movenet_prob = movenet_model.predict_proba(X)[0]
        #print(movenet_class, movenet_prob)
        
    # Get status box
        cv2.rectangle(frame, (0,0), (250, 60), (245, 117, 16), -1)
        
    # Display Class
        cv2.putText(frame, 'CLASS'
                    , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(frame, movenet_class.split(' ')[0]
                    , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
    # Display Probability
        cv2.putText(frame, 'PROB'
                    , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(frame, str(round(movenet_prob[np.argmax(movenet_prob)],2))
                    , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)   
        
    #Visualise
        cv2.putText(frame , str(int(neck_inclination)), tuple(np.multiply(left_shoulder, [1850,800]).astype(int)), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255), 2, cv2.LINE_AA)
        cv2.putText(frame, str(int(left_hip_angle)), tuple(np.multiply(left_hip, [1850,1100]).astype(int)), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255), 2, cv2.LINE_AA)
        #cv2.putText(frame, str(int(left_knee_angle)), tuple(np.multiply(left_knee, [1850,800]).astype(int)), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255), 2, cv2.LINE_AA)
        cv2.putText(frame, str(int(left_bicep_angle)), tuple(np.multiply(left_elbow, [1850,1100]).astype(int)), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255), 2, cv2.LINE_AA)
    except:
        pass
    
    # Render keypoints 
    loop_through_people(frame, keypoints_with_scores, EDGES, 0.3)
    
    cv2.imshow('Movenet Multipose', frame)
    
    if cv2.waitKey(10) & 0xFF==ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

In [14]:
cap = cv2.VideoCapture('http://192.168.1.23:8080/video')
while cap.isOpened():
    ret, frame = cap.read()
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    # Resize image
    img = frame.copy()
    img = tf.image.resize_with_pad(tf.expand_dims(img, axis=0), 192,320)
    input_img = tf.cast(img, dtype=tf.int32)
    
    # Detection section
    results = movenet(input_img)
    keypoints_with_scores = results['output_0'].numpy()[:,:,:51].reshape((6,17,3))
    
    try:
        left_shoulder_x = keypoints_with_scores[0][5][1]
        left_shoulder_y = keypoints_with_scores[0][5][0]
        left_ear_x = keypoints_with_scores[0][3][1]
        left_ear_y = keypoints_with_scores[0][3][0]
        
        right_shoulder_x = keypoints_with_scores[0][6][1]
        right_shoulder_y = keypoints_with_scores[0][6][0]
        right_ear_x = keypoints_with_scores[0][4][1]
        right_ear_y = keypoints_with_scores[0][4][0]
        
        left_shoulder = [left_shoulder_x, left_shoulder_y]
        
        left_hip = [keypoints_with_scores[0][11][1], keypoints_with_scores[0][11][0]]
        left_elbow = [keypoints_with_scores[0][7][1], keypoints_with_scores[0][7][0]]
        left_knee = [keypoints_with_scores[0][13][1], keypoints_with_scores[0][13][0]]
        left_ankle = [keypoints_with_scores[0][15][1], keypoints_with_scores[0][15][0]]
        left_wrist = [keypoints_with_scores[0][9][1], keypoints_with_scores[0][9][0]]
        left_ear = [keypoints_with_scores[0][3][1], keypoints_with_scores[0][3][0]]
        
        right_hip = [keypoints_with_scores[0][12][1], keypoints_with_scores[0][12][0]]
        right_shoulder = [keypoints_with_scores[0][6][1], keypoints_with_scores[0][6][0]]
        right_elbow = [keypoints_with_scores[0][8][1], keypoints_with_scores[0][8][0]]
        right_knee = [keypoints_with_scores[0][14][1], keypoints_with_scores[0][14][0]]
        right_ankle = [keypoints_with_scores[0][16][1], keypoints_with_scores[0][16][0]]
        right_wrist = [keypoints_with_scores[0][10][1], keypoints_with_scores[0][10][0]]
        right_ear = [keypoints_with_scores[0][5][1], keypoints_with_scores[0][5][0]]
        
        neck_inclination = calculate_angle2(left_shoulder_x, left_shoulder_y, left_ear_x, left_ear_y)
        
        left_hip_angle = calculate_angle(left_shoulder, left_hip, left_knee)
        #left_knee_angle = calculate_angle(left_hip, left_knee, left_ankle)
        left_bicep_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
        
        right_hip_angle = calculate_angle(right_shoulder, right_hip, right_knee)
        #right_knee_angle = calculate_angle(right_hip, right_knee, right_ankle)
        right_bicep_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)
        #print(left_bicep_angle)
        #print(neck_inclination)
        #print(left_shoulder)
    except:
        pass
    
    #Export coordinates
    try:
        pose_row =list(keypoints_with_scores[0].flatten())
        
        left_bicep_row = list(np.array(left_bicep_angle).flatten())
        left_hip_row = list(np.array(left_hip_angle).flatten())
        
        right_bicep_row = list(np.array(right_bicep_angle).flatten())
        right_hip_row = list(np.array(right_hip_angle).flatten())
        
        neck_inclination_row = list(np.array(neck_inclination).flatten())
        
        row = neck_inclination_row + left_hip_row + left_bicep_row + right_hip_row + right_bicep_row + pose_row
        #row.insert(0,class_name)
        
        #with open('coords_movenet_lieback.csv', mode ='a', newline ='') as f:
                #csv_writer = csv.writer(f, delimiter =',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                #csv_writer.writerow(row)
                
    #Make Detections and Predictions
        X = pd.DataFrame([row])
        movenet_class = movenet_model.predict(X)[0]
        movenet_prob = movenet_model.predict_proba(X)[0]
        #print(movenet_class, movenet_prob)
        
    # Get status box
        cv2.rectangle(frame, (0,0), (250, 60), (245, 117, 16), -1)
        cv2.rectangle(frame, (0,1000), (250, 1100), (245, 117, 16), -1)
        cv2.rectangle(frame, (2000,1000), (1550, 1100), (245, 117, 16), -1)
            
#Display Good vs Bad Posture
        if movenet_class == 'straight':
            bad_posture = 0
            good_posture += 1
            cv2.putText(frame, 'CLASS'
                        , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(frame, movenet_class.split(' ')[0]
                        , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (127, 233, 100), 2, cv2.LINE_AA)
            cv2.putText(frame, 'Good Posture'
                        , (15,1040), cv2.FONT_HERSHEY_SIMPLEX,  1, (127, 233, 100), 2, cv2.LINE_AA)
            cv2.putText(frame, 'PROB'
                        , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(frame, str(round(movenet_prob[np.argmax(movenet_prob)],2))
                        , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (127, 233, 100), 2, cv2.LINE_AA)
        else:
            good_posture = 0
            bad_posture += 1 
            cv2.putText(frame, 'CLASS'
                        , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(frame, movenet_class.split(' ')[0]
                        , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (50, 50, 255), 2, cv2.LINE_AA)
            cv2.putText(frame, 'Bad Posture'
                        , (15,1040), cv2.FONT_HERSHEY_SIMPLEX,  1, (50, 50, 255), 2, cv2.LINE_AA)
            cv2.putText(frame, 'PROB'
                        , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(frame, str(round(movenet_prob[np.argmax(movenet_prob)],2))
                        , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (50, 50, 255), 2, cv2.LINE_AA)
     
    #Getting pose time
        good_time_posture = (2.2 / fps) * good_posture
        bad_time_posture = (2.2 / fps) * bad_posture
            
        if good_time_posture > 0:
            good_time_posture_print = 'Good Posture Time: ' + str(round(good_time_posture,1)) + 's'
            cv2.putText(frame, good_time_posture_print
                        , (1600,1040), cv2.FONT_HERSHEY_SIMPLEX,  0.7, (127, 233, 100), 2, cv2.LINE_AA)
        else:
            bad_time_posture_print = 'Bad Posture Time: ' + str(round(bad_time_posture,1)) + 's'
            cv2.putText(frame, bad_time_posture_print
                        , (1600,1040), cv2.FONT_HERSHEY_SIMPLEX,  0.7, (50, 50, 255), 2, cv2.LINE_AA)
            
        if bad_time_posture > 20:
            sendWarning()
        
    #Visualise
        cv2.putText(frame , str(int(neck_inclination)), tuple(np.multiply(left_shoulder, [1850,800]).astype(int)), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255), 2, cv2.LINE_AA)
        cv2.putText(frame, str(int(left_hip_angle)), tuple(np.multiply(left_hip, [1850,1100]).astype(int)), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255), 2, cv2.LINE_AA)
        #cv2.putText(frame, str(int(left_knee_angle)), tuple(np.multiply(left_knee, [1850,800]).astype(int)), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255), 2, cv2.LINE_AA)
        cv2.putText(frame, str(int(left_bicep_angle)), tuple(np.multiply(left_elbow, [1850,1100]).astype(int)), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255), 2, cv2.LINE_AA)
    except:
        pass
    
    # Render keypoints 
    loop_through_people(frame, keypoints_with_scores, EDGES, 0.3)
    
    cv2.imshow('LiveCam Feed', frame)
    
    if cv2.waitKey(10) & 0xFF==ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

### Model Limitation and Conclusion

The best models (Random Forest Classifier and Extra Tree Classifier from PyCaret) are able to detect various pre-determined postures based on pose estimation model for both webcam and sidecam. Summary of models are as follows:

**Summary of models are as follows (for webcam):**

|Models|Validation Accuracy|Mean F1 Score of 5 Postures|
|---|:--:|:--:|
|Logistic Regression|99.8%|99.8%|
|Ridge Classification|97.2%|97.2%|
|Random Forest Classifier|99.6%|99.6%|
|Gradient Boosting Classifier|99.6%|99.8%|
|Adaptive Boosting (ADA) Classifier|53.2%|45.1%|
|Decision Tree Classifier|98.6%|98.6%|

**Summary of models are as follows (for sidecam):**

|Models|Validation Accuracy|Mean F1 Score of 5 Postures|
|---|:--:|:--:|
|Logistic Regression|98.8%|98.8%|
|Ridge Classification|95.2%|94.5%|
|Random Forest Classifier|99.6%|99.1%|
|Gradient Boosting Classifier|99.5%|99%|
|Adaptive Boosting (ADA) Classifier|59.4%|57.7%|
|Decision Tree Classifier|97.4%|97%|

Although current model has performed well under real live camera, it has some limitations:
    
1. Model is currently limited to identifying straight, slouched/hunched, slouched/hunched + roundshoulders, uneven shoulder(webcam) and lieback. However, there are many combined postures such as cross legs, butterfly sit posture, knee angle detection, etc. In addition, there are many more complex postures and alignments such as monitor alignment, table/chair height optimisation, etc.


2. Current models are trained based on limited own dataset with certain angles and camera placements. In the future, more cameras can be used to train to create more robust models to cater for all environments.


3. Model is only capable of reminding bad postures after 15 secs through audio. However, in real life scenario, it is better to give live angle recommendations for the users to adjust to straight posture.


 All in all, future improvements can include :
 - More complex postures specifically legs joints as these body parts also affect sitting ergonomics.
 
 
 - Utilise more cameras to cover all camera angles/placements for model training. Thus, it will be able to create a much robust model at different placements when deployed to offices/homes.
 
 
 - Train more models to cater for 2 or more people in a frame.
 
 
 - Explore deep learning models (CNN or RNN) for dataset training.
 
 
 - It is also possible to use object detection models such as Mask R-CNN or YOLO to detect monitors, chairs and tables to ultimately give recommendations for best alignments/placements and heights for each workstation.
 
 
 - Use Tkinter to develop GUI app for posture record and recommendations to correct postures.